# Get closest weather stations to address

The Geocoding (transforming a postal address description to a location on the Earth's surface) is based on `pygeocoder`. Install it by running `pip install pygeocoder`.

In [110]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [306]:
import geocoding_functions as geo_fns
import location_functions as loc_fns

## First examples

An example of a valid and invalid address. Get coodrinates if address is valid

In [127]:
invalid_address = "invalid"
valid_address = "Avenue du Temple 3, Renens, Switzerland"

In [131]:
invalid_coordinates = geo_fns.getCoordinates(invalid_address)

In [130]:
coordinates = geo_fns.getCoordinates(valid_address)

In [132]:
coordinates

(46.5364118, 6.5877215)

In [133]:
print(invalid_coordinates)

(0, 0)


Testing the method for getting a distance between two points

In [134]:
loc_fns.get_distance(coordinates, (0,0))

5214363.3588638557

In [135]:
loc_fns.get_distance(invalid_coordinates, (0,0))

0.0

## Testing with Swiss addresses

The following are locations of some restaurants from Google Maps.

In [136]:
restaurants_list = ["Avenue Emile-Henri-Jaques-Dalcroze 9, 1007 Lausanne",
                   "Rue d'Yverdon 1, 1023 Crissier",
                   "Rue du Village 10, 1034 Boussens",
                   "Kornhauspl. 18, 3011 Bern",
                   "Talgut-Zentrum 7, 3063 Ittigen",
                   "Neumühlequai 42, 8006 Zürich",
                   "Via ai Grotti 71, 6652 Ponte Brolla, Tegna"]

In [138]:
coordinates_list = []
for restaurant in restaurants_list:
    coordinates_list.append(geo_fns.getCoordinates(restaurant))

In [139]:
coordinates_list

[(46.5118279, 6.6093473),
 (46.5540867, 6.576671699999999),
 (46.604882, 6.5830395),
 (46.9488427, 7.4472494),
 (46.9743359, 7.483379899999999),
 (47.38255789999999, 8.5405114),
 (46.189274, 8.7547559)]

## Get all station names and locations

In [286]:
import station_info
import pandas as pd
import pickle

In [243]:
stationsFile = 'stations.txt'

In [244]:
station_dict = station_info.getStationNamesAndLocations(stationsFile)

In [245]:
station_df = pd.DataFrame.from_dict(station_dict, orient='index')
station_df.reset_index(inplace=True)
station_df = station_df.rename(columns = {'index':'name', 0:'lat', 1:'lng'})
station_df.head()

,name,lat,lng
0,VSVER,46.233333,7.550000
1,VSCOL,46.183333,7.383333
2,AIG,46.333333,6.916667
3,DEM,47.350000,7.350000
4,SRS,46.983333,9.666667


In [288]:
pickle.dump(station_df, open("station_df.p", "wb"))

## Forming stations list to get nearest stations

In [246]:
allLat = station_df['lat'].get_values()
allLng = station_df['lng'].get_values()
stations = list(zip(allLat, allLng))

In [289]:
pickle.dump(stations, open("stations_lat_lng.p", "wb"))

### Testing the method for getting the nearest station for given coordinates

In [259]:
coordinates = geo_fns.getCoordinates("Route Cantonale 39B, 1025 St. Sulpice")
coordinates

(46.5159793, 6.5637658)

In [260]:
# returns a tuple (distance, stateID)
nearest_station = loc_fns.getNearestStation(coordinates, stations)
nearest_station_distance = nearest_station[0]
nearest_station_index = nearest_station[1]
station_df.name[nearest_station_index]

'WSLLAF'

Testing the method for returning the list of stations sorted by their distance to the given coordinates<br>
<b>TODO:</b> Use this method to choose the number of stations to use, and the weight assigned to each

In [280]:
import numpy as np

In [281]:
# returns a sorted list of (distance, stationID) tuples
sorted_stations = loc_fns.getStationDistances(coordinates_list[6], stations)

In [292]:
loc_fns.get_k_nearest_neighbours(4, sorted_stations, station_df)

25       CIM
82       OTL
99       MAG
14    WSLNOF
Name: name, dtype: object

In [294]:
loc_fns.get_weights_for_k_nearest(4, sorted_stations)

array([ 0.46120978,  0.3863496 ,  0.07820982,  0.07423079])

### Parameters

In [308]:
loc_fns.get_k_nearest_neighbours(coordinates, 4, station_df)

44     WSLLAF
26        PUY
41     WSLLAB
144       VIT
Name: name, dtype: object